In [3]:
import numpy as np
from keras.models import Model
from keras.layers import Input,LSTM,Dense

In [4]:
#Define the input sequence length and dimensionlity
input_seq_length=10
input_dim=20

In [5]:
#Define the output sequence length and dimensionality
output_seq_length=5
out_dim=10

In [6]:
#Define the number of units in the LSTM layers
latent_dim=256

In [7]:
#Define the input layer
encoder_input=Input(shape=(input_seq_length,input_dim))

In [9]:
#Define the encoder LSTM layer
encoder_lstm=LSTM(latent_dim,return_state=True)#only return last state

In [10]:
#Get the encoder outputs and states
encoder_outputs,state_h,state_c=encoder_lstm(encoder_input)

In [11]:
#Keep only the states
encoder_states=[state_h,state_c]

In [12]:
#Define the decoder input layer
decoder_input=Input(shape=(output_seq_length,out_dim))

In [13]:
#Define the decoder LSTM layer, wit the encoder states as initial state
decoder_lstm=LSTM(latent_dim,return_sequences=True,return_state=True)

In [20]:
#Get the decoder outputs
decoder_output,x,y=decoder_lstm(decoder_input,initial_state=encoder_states)

In [21]:
#Define the output layer
decoder_dense=Dense(out_dim, activation='softmax')
decoder_outputs=decoder_dense(decoder_output)

In [22]:
#Define the model
model=Model([encoder_input,decoder_input],decoder_outputs)

In [23]:
#Compile the model
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [24]:
#Print model summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 10, 20)]             0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 5, 10)]              0         []                            
                                                                                                  
 lstm_1 (LSTM)               [(None, 256),                283648    ['input_1[0][0]']             
                              (None, 256),                                                        
                              (None, 256)]                                                        
                                                                                              

In [26]:
#Generating dummy training data
X_train_encoder = np.random.randn(1000, input_seq_length, input_dim)
X_train_decoder = np.random.randn(1000, output_seq_length, out_dim)
y_train = np.random.randn(1000, output_seq_length, out_dim)

In [29]:
#Train the model
model.fit([X_train_encoder,X_train_decoder],y_train,epochs=50,batch_size=32)

Epoch 1/50
32/32 [==============================] - 1s 40ms/step - loss: -0.1116 - accuracy: 0.1058
Epoch 2/50
32/32 [==============================] - 1s 39ms/step - loss: -0.1079 - accuracy: 0.1052
Epoch 3/50
32/32 [==============================] - 1s 39ms/step - loss: -0.1041 - accuracy: 0.1038
Epoch 4/50
32/32 [==============================] - 1s 40ms/step - loss: -0.1291 - accuracy: 0.0942
Epoch 5/50
32/32 [==============================] - 1s 40ms/step - loss: -0.1031 - accuracy: 0.1032
Epoch 6/50
32/32 [==============================] - 1s 39ms/step - loss: -0.1031 - accuracy: 0.0924
Epoch 7/50
32/32 [==============================] - 2s 64ms/step - loss: -0.1163 - accuracy: 0.0970
Epoch 8/50
32/32 [==============================] - 2s 57ms/step - loss: -0.1123 - accuracy: 0.0990
Epoch 9/50
32/32 [==============================] - 1s 40ms/step - loss: -0.1146 - accuracy: 0.1044
Epoch 10/50
32/32 [==============================] - 1s 39ms/step - loss: -0.1085 - accuracy: 0.1006

In [31]:
# Generate dummy test data
X_test_encoder = np.random.randn(200, input_seq_length, input_dim)
X_test_decoder = np.random.randn(200, output_seq_length, out_dim)

In [32]:
# Predict using the trained model
predictions = model.predict([X_test_encoder, X_test_decoder])
print("Predictions shape:", predictions.shape)

7/7 [==============================] - 2s 22ms/step
Predictions shape: (200, 5, 10)
